In [1]:
import xarray as xr
import numpy as np
import cftime
import nc_time_axis
import pandas as pd
import matplotlib.pyplot as plt 
from matplotlib import ticker, cm
#from cftime import datetime 
from datetime import datetime, timedelta
#import datetime as dt
import matplotlib.colors as colors
import math
import random
import matplotlib.cm as mcm
from matplotlib.ticker import MaxNLocator
#jet = mcm.get_cmap('jet')
jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')
import netCDF4 as nc
import sys
import os
import psutil
import netCDF4 as nc
import cartopy.crs as ccrs # CRS stands for "Coordinate reference systems" for map projection
from cartopy.crs import PlateCarree
import cartopy.feature as cfeature
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from dateutil import tz
import pytz
import time
from time import process_time
from tqdm import tqdm
from dask.distributed import LocalCluster, Client
import dask.array as da
import dask.dataframe as dd
import dask
from dask import delayed
from dask.diagnostics import ProgressBar  # Import the ProgressBar class
import calendar
%matplotlib inline 
#import line_profiler
#%load_ext line_profiler

/tmp/ipykernel_3972062/2706121075.py:17: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed two minor releases later. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap(obj)`` instead.
  jet = mcm.get_cmap('jet') if isinstance(mcm.get_cmap('jet'), str) else mcm.get_cmap('jet')


In [2]:




fivedegbins = 1






In [4]:
################################################################################
# Open WACCM dataset to define bins
################################################################################


# Define the relative path to the symbolic link
symbolic_link = 'Nc_Files/ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport'

# Get the full path to the symbolic link
link_path = os.path.expanduser(os.path.expandvars(os.path.normpath(symbolic_link)))

# Join the symbolic link with the specific file you want to open
file_name = 'ACP_CESM213_FX2000_f19_f19_mg16_Na_Fe_Mg_iontransport.cam.h2.0001-01-01-00000.nc'
file_path = os.path.join(link_path, file_name)

# Open the file
wds = xr.open_dataset(file_path)
#wds

In [5]:
wds_lat = wds['lat']
wds_lon = wds['lon']
wds_lev = wds['lev']
wds_ilev = wds['ilev']
wds_geopH = wds.variables['Z3'] / 1000 #m-> km
wds_Zavg = wds_geopH.mean(('time','lat', 'lon'))
wds_time = wds['time']

# Slice arrays (lev, altitude) between chosen range (~80-130km)
lev_sl_idx_min = 42
lev_sl_idx_max = 66

# Slice arrays (lev & alt) between chosen range & print
lev_sl = wds_lev[lev_sl_idx_min:lev_sl_idx_max+1]
Zavg_sl = wds_Zavg[lev_sl_idx_min:lev_sl_idx_max+1]


print('    Array lev = ' + str("%.1e" % wds_lev[lev_sl_idx_max] ) + 'hPa : ' + str("%.1e" % wds_lev[lev_sl_idx_min] ) + 'hPa'
           + ' (approx ' + str("%.0f" % wds_Zavg[lev_sl_idx_max]) + 'km : ' + str("%.0f" % wds_Zavg[lev_sl_idx_min] ) + 'km' + ')'   )


    Array lev = 6.0e-03hPa : 1.5e-05hPa (approx 81km : 126km)


In [6]:
################################################################################
# Lat bins
################################################################################

lat_values = wds_lat.values

lat_bin_edges = lat_values
lat_bin_midpoints = (lat_bin_edges[:-1] + lat_bin_edges[1:]) / 2
#lat_bin_widths = np.diff(lat_bin_edges)
#lat_average_bin_width = np.mean(lat_bin_widths)
lat_num_bins = len(lat_bin_midpoints)


# # ##Print the results
# print("Lat Bin Edges (Lower and Upper):")
# for i in range(len(lat_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lat_bin_edges[i]}, {lat_bin_edges[i + 1]}]")

# print("\nLat Bin Midpoints:")
# for i in range(len(lat_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {lat_bin_midpoints[i]}")

# print("\nLat Bin Widths:", lat_bin_widths)
# print(f"Lat Average Bin Width: {lat_average_bin_width:.6f} degrees")
# print(f"Lat Number of Bins: {lat_num_bins}")


# # Lat Average Bin Width: 1.894737 degrees
# # Lat Number of Bins: 95

In [7]:
# ################################################################################
# # Lon bins -180 -> 180
# ################################################################################

# Updated value for a range of -180 to +180 degrees
lon_upper_edge_last_bin = 180.0
lon_bin_edges = np.append(np.arange(-180.0, 180.0, 2.5), lon_upper_edge_last_bin)
lon_bin_midpoints = (lon_bin_edges[:-1] + lon_bin_edges[1:]) / 2

# Other calculations (width, number of bins, etc.) remain the same
lon_bin_widths = np.diff(lon_bin_edges)
lon_average_bin_width = np.mean(lon_bin_widths)
lon_num_bins = len(lon_bin_midpoints)

# # Print the results
# print("Lon Bin Edges (Lower and Upper):")
# for i in range(len(lon_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{lon_bin_edges[i]}, {lon_bin_edges[i + 1]}]")

# print("\nLon Bin Midpoints:")
# for i in range(len(lon_bin_midpoints)):
#     print(f"Lon Bin {i + 1} Midpoint: {lon_bin_midpoints[i]}")

# print("\nLon Bin Widths:", lon_bin_widths)
# print(f"Average Lon Bin Width: {lon_average_bin_width:.8f} degrees")
# print(f"Number of Lon Bins: {lon_num_bins}")

In [8]:
################################################################################
# 5 deg Lat bins
################################################################################

if fivedegbins == 1:
    lat_values = np.arange(-90, 95, 5)
    lat_bin_edges = lat_values
    lat_bin_midpoints = (lat_bin_edges[:-1] + lat_bin_edges[1:]) / 2
    lat_num_bins = len(lat_bin_midpoints)

    lon_bin_edges = np.arange(-180, 185, 5)
    lon_bin_midpoints = (lon_bin_edges[:-1] + lon_bin_edges[1:]) / 2
    lon_num_bins = len(lon_bin_midpoints) #72

In [9]:
################################################################################
# Zavg_sl bins - Note first bin is at top
################################################################################

# Convert xarray Variable to NumPy array
Zavg_sl_values = Zavg_sl.values

Zavg_sl_bin_widths = np.diff(Zavg_sl_values)
Zavg_sl_bin_edges = [Zavg_sl_values[0] - Zavg_sl_bin_widths[0] / 2] + [Zavg_sl_values[i] + Zavg_sl_bin_widths[i] / 2 for i in range(len(Zavg_sl_bin_widths))] + [Zavg_sl_values[-1] + Zavg_sl_bin_widths[-1] / 2]
Zavg_sl_bin_midpoints = [(Zavg_sl_bin_edges[i] + Zavg_sl_bin_edges[i + 1]) / 2 for i in range(len(Zavg_sl_bin_edges) - 1)]
#Zavg_sl_bin_widths = [Zavg_sl_bin_edges[i + 1] - Zavg_sl_bin_edges[i] for i in range(len(Zavg_sl_bin_edges) - 1)]
Zavg_sl_num_bins = len(Zavg_sl_bin_midpoints)


# # Print the bin edges and midpoints
# print("Zavg_sl Bin Edges (Lower and Upper):")
# for i in range(len(Zavg_sl_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{Zavg_sl_bin_edges[i]}, {Zavg_sl_bin_edges[i + 1]}")

# print("\nZavg_sl Bin Midpoints:")
# for i in range(len(Zavg_sl_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {Zavg_sl_bin_midpoints[i]}")

# # Print the bin widths
# print("\nZavg_sl Bin Widths:")
# for i in range(len(Zavg_sl_bin_widths)):
#     print(f"Bin {i + 1} Width: {Zavg_sl_bin_widths[i]}")

# print(f"Number of Zavg_sl_bin_widths Bins: {Zavg_sl_num_bins}")

#Number of Zavg_sl_bin_widths Bins: 25

In [10]:
LT_bin_edges = np.linspace(0, 24, 49)
LT_bin_edges[-1] = 24.0

# Calculate bin midpoints
LT_bin_midpoints = (LT_bin_edges[:-1] + LT_bin_edges[1:]) / 2

# Calculate the number of bins
LT_num_bins = len(LT_bin_edges) - 1

# Calculate bin widths
LT_bin_widths = np.diff(LT_bin_edges)

# Calculate the average bin width
average_bin_width = np.sum(LT_bin_widths) / LT_num_bins


# ##Print the results
# print("LT Bin Edges (Lower and Upper):")
# for i in range(len(LT_bin_edges) - 1):
#     print(f"Bin {i + 1}: [{LT_bin_edges[i]}, {LT_bin_edges[i + 1]}]")

# print("\nLT Bin Midpoints:")
# for i in range(len(LT_bin_midpoints)):
#     print(f"Bin {i + 1} Midpoint: {LT_bin_midpoints[i]}")

# print("\nNumber of LT Bins:")
# print(LT_num_bins)

# # Print the average bin width
# print("Average LT Bin Width:", average_bin_width)

In [12]:
### Raw satellite data ###
filepath = 'Nc_Files/s4max_files/mod_s4max.nc'

### Processed time data into datetime format ###
#new_filepath = 'Nc_Files/s4max_files/mod_s4max.nc'

### My processed data - 80-130km only ###
#result_filepath = 'Nc_Files/s4max_files/mod_s4max_80-130km.nc'

ds = xr.open_dataset(filepath)
ds

<xarray.Dataset>
Dimensions:       (nfile: 9266536)
Coordinates:
    altitude      (nfile) float64 ...
    lat           (nfile) float64 ...
    lon           (nfile) float64 ...
    time          (nfile) datetime64[ns] ...
    season        (nfile) object ...
    month         (nfile) object ...
Dimensions without coordinates: nfile
Data variables: (12/14)
    Ut_time       (nfile) float64 ...
    DOY           (nfile) float64 ...
    LT            (nfile) float64 ...
    mlat          (nfile) float64 ...
    mlon          (nfile) float64 ...
    mlt           (nfile) float64 ...
    ...            ...
    month_var     (nfile) object ...
    season_var    (nfile) object ...
    lat_var       (nfile) float64 ...
    lon_var       (nfile) float64 ...
    altitude_var  (nfile) float64 ...
    time_var      (nfile) datetime64[ns] ...

In [17]:
time_test = ds['time_var']
# time_test[0].values  #numpy.datetime64('2006-12-28T21:56:00.000000000')
# time_test[-1].values   #numpy.datetime64('2014-01-30T23:26:00.000000000')

# --> 2007 to 2013 full years

# Define your date range
start_date = np.datetime64('2007-03-01')
end_date = np.datetime64('2013-11-30')

# Create a boolean mask for the date range
mask = (time_test >= start_date) & (time_test <= end_date)

# Apply the mask to get the subset of data between the two dates
subset_time_test = time_test[mask]

# subset_time_test[0].values  #numpy.datetime64('2007-03-01T00:39:00.000000000')
# subset_time_test[-1].values   #numpy.datetime64('2013-11-30T00:00:00.000000000')

numpy.datetime64('2014-01-30T23:26:00.000000000')

In [ ]:
ds_sub = ds.where(mask, drop=True)

# Now, use the modified 'time' coordinate to select the subset of the dataset
ds_sub = ds.sel(time=ds_sub['time'])
ds_sub

In [26]:
ds.sel(time=slice(start_date, end_date) )

KeyError: "no index found for coordinate 'time'"

In [11]:
####################################      N O R M A L  Lat Lon coordinates     ##############################################

s4max_threshold = 0.35

#Arras and Wickert 2018 s4max > 0.2
#Yue et al 2016 s4max > 0.3
#Yue et al 2015 s4max > 0.3
#Ray 2006 s4max > 0.4
#Ko and Yeh 2010 'strong scintillations' defined as smax >= 0.5

###########################################################################################################################



# Define multiple sets of desired months
all_desired_months = [ [3, 4, 5] ] #, [6, 7, 8], [9, 10, 11] , [12, 1, 2] ]
    
unique_years = range(2007, 2014)
for year in unique_years:
    print(f"Analyzing data for year {year}")
    
    # Filter data for the current year
    year_mask = ds['year'] == year

    




    
# Loop over each set of desired months
for desired_months in all_desired_months:
    print('Months ' + str(desired_months))
    averaging_period = ""
    if len(desired_months) == 1:
        averaging_period = "Monthly"
    elif len(desired_months) == 2:
        averaging_period = "Two-Month"
    elif len(desired_months) == 3:
        averaging_period = "Three-Month"
    else:
        averaging_period = "Unknown"

    season = ""
    if set(desired_months) <= {12, 1, 2}:
        season = "winter"
    elif set(desired_months) <= {3, 4, 5}:
        season = "spring"
    elif set(desired_months) <= {6, 7, 8}:
        season = "summer"
    elif set(desired_months) <= {9, 10, 11}:
        season = "autumn"
    else:
        season = "unknown"

    freq_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))
    nmeas_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))

    freq_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))
    nmeas_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))

    nfile = len(ds['lat'])
    ds_length = np.arange(nfile)

    ###########################################################################################################################
    for i in tqdm(ds_length):
        current_altitude = ds['altitude_var'].values[i]
        current_lat = ds['lat'].values[i]
        current_lon = ds['lon'].values[i]
        current_LT = ds["LT"].values[i]
        current_s4max = ds['s4max'].values[i]

        current_time = ds["time"].values[i]
        current_month = int(np.datetime64(current_time, 'M').astype('M8[M]').view('i8')) % 12 + 1

        if current_month in desired_months:
            if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                if current_lon == lon_bin_edges[-1]:
                    lon_bin_index = len(lon_bin_edges) - 2
                LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                if current_LT == LT_bin_edges[-1]:
                    LT_bin_index = len(LT_bin_edges) - 2

                nmeas_lon[altitude_bin_index, lat_bin_index, lon_bin_index] += 1.
                nmeas_LT[altitude_bin_index, lat_bin_index, LT_bin_index] += 1.

                if current_s4max > s4max_threshold:
                    freq_lon[altitude_bin_index, lat_bin_index, lon_bin_index] += 1.
                    freq_LT[altitude_bin_index, lat_bin_index, LT_bin_index] += 1.

    ###########################################################################################################################
    # Calculate occurence frequency
    ocfr_lon = np.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins), 0.)
    ocfr_LT = np.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins), 0.)

    # Check if arrays are equal to 0
    lon_mask = (freq_lon != 0.) & (nmeas_lon != 0.)
    LT_mask = (freq_LT != 0.) & (nmeas_LT != 0.)

    # Calculate ocfr_lon and ocfr_LT only where the mask is True
    if lon_mask.any():
        ocfr_lon[lon_mask] = (freq_lon[lon_mask] / nmeas_lon[lon_mask]) * 100.0

    if LT_mask.any():
        ocfr_LT[LT_mask] = (freq_LT[LT_mask] / nmeas_LT[LT_mask]) * 100.0

    ###########################################################################################################################
    # Save occurrence frequency to nc file
    Ocfr_ds = xr.Dataset(
        data_vars={
            "ocfr_lon": (['altitude', 'latitude', 'longitude'], ocfr_lon),
            "ocfr_LT": (['altitude', 'latitude', 'local_time'], ocfr_LT)
        },
        coords={
            "altitude": Zavg_sl_bin_midpoints,
            "latitude": lat_bin_midpoints,
            "longitude": lon_bin_midpoints,
            "local_time": LT_bin_midpoints,
        },
        attrs={
            'averaging_period': averaging_period,
            'season': season
        }
    )

    output_directory = "./Nc_Files/s4max/"
    output_file = f"{output_directory}Ocfr_{season}_s4max_{s4max_threshold}.nc"
    if fivedegbins == 1:
        output_file = f"{output_directory}Ocfr_{season}_5d_s4max_{s4max_threshold}.nc"
    Ocfr_ds.to_netcdf(output_file)

###########################################################################################################################


Months [3, 4, 5]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:53<00:00, 12991.03it/s]


Months [6, 7, 8]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:49<00:00, 13060.29it/s]


Months [9, 10, 11]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:39<00:00, 13252.85it/s]


Months [12, 1, 2]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:45<00:00, 13136.09it/s]


In [11]:
####################################      N O R M A L  Lat Lon coordinates     ##############################################

s4max_threshold = 0.35

#Arras and Wickert 2018 s4max > 0.2
#Yue et al 2016 s4max > 0.3
#Yue et al 2015 s4max > 0.3
#Ray 2006 s4max > 0.4
#Ko and Yeh 2010 'strong scintillations' defined as smax >= 0.5

###########################################################################################################################
# Define multiple sets of desired months
all_desired_months = [ [3, 4, 5], [6, 7, 8], [9, 10, 11] , [12, 1, 2] ]

# Loop over each set of desired months
for desired_months in all_desired_months:
    print('Months ' + str(desired_months))
    averaging_period = ""
    if len(desired_months) == 1:
        averaging_period = "Monthly"
    elif len(desired_months) == 2:
        averaging_period = "Two-Month"
    elif len(desired_months) == 3:
        averaging_period = "Three-Month"
    else:
        averaging_period = "Unknown"

    season = ""
    if set(desired_months) <= {12, 1, 2}:
        season = "winter"
    elif set(desired_months) <= {3, 4, 5}:
        season = "spring"
    elif set(desired_months) <= {6, 7, 8}:
        season = "summer"
    elif set(desired_months) <= {9, 10, 11}:
        season = "autumn"
    else:
        season = "unknown"

    freq_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))
    nmeas_LT = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))

    freq_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))
    nmeas_lon = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))

    nfile = len(ds['lat'])
    ds_length = np.arange(nfile)

    ###########################################################################################################################
    for i in tqdm(ds_length):
        current_altitude = ds['altitude_var'].values[i]
        current_lat = ds['lat'].values[i]
        current_lon = ds['lon'].values[i]
        current_LT = ds["LT"].values[i]
        current_s4max = ds['s4max'].values[i]

        current_time = ds["time"].values[i]
        current_month = int(np.datetime64(current_time, 'M').astype('M8[M]').view('i8')) % 12 + 1

        if current_month in desired_months:
            if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                lat_bin_index = np.digitize(current_lat, lat_bin_edges) - 1
                lon_bin_index = np.digitize(current_lon, lon_bin_edges) - 1
                if current_lon == lon_bin_edges[-1]:
                    lon_bin_index = len(lon_bin_edges) - 2
                LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                if current_LT == LT_bin_edges[-1]:
                    LT_bin_index = len(LT_bin_edges) - 2

                nmeas_lon[altitude_bin_index, lat_bin_index, lon_bin_index] += 1.
                nmeas_LT[altitude_bin_index, lat_bin_index, LT_bin_index] += 1.

                if current_s4max > s4max_threshold:
                    freq_lon[altitude_bin_index, lat_bin_index, lon_bin_index] += 1.
                    freq_LT[altitude_bin_index, lat_bin_index, LT_bin_index] += 1.

    ###########################################################################################################################
    # Calculate occurence frequency
    ocfr_lon = np.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins), 0.)
    ocfr_LT = np.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins), 0.)

    # Check if arrays are equal to 0
    lon_mask = (freq_lon != 0.) & (nmeas_lon != 0.)
    LT_mask = (freq_LT != 0.) & (nmeas_LT != 0.)

    # Calculate ocfr_lon and ocfr_LT only where the mask is True
    if lon_mask.any():
        ocfr_lon[lon_mask] = (freq_lon[lon_mask] / nmeas_lon[lon_mask]) * 100.0

    if LT_mask.any():
        ocfr_LT[LT_mask] = (freq_LT[LT_mask] / nmeas_LT[LT_mask]) * 100.0

    ###########################################################################################################################
    # Save occurrence frequency to nc file
    Ocfr_ds = xr.Dataset(
        data_vars={
            "ocfr_lon": (['altitude', 'latitude', 'longitude'], ocfr_lon),
            "ocfr_LT": (['altitude', 'latitude', 'local_time'], ocfr_LT)
        },
        coords={
            "altitude": Zavg_sl_bin_midpoints,
            "latitude": lat_bin_midpoints,
            "longitude": lon_bin_midpoints,
            "local_time": LT_bin_midpoints,
        },
        attrs={
            'averaging_period': averaging_period,
            'season': season
        }
    )

    output_directory = "./Nc_Files/s4max/"
    output_file = f"{output_directory}Ocfr_{season}_s4max_{s4max_threshold}.nc"
    if fivedegbins == 1:
        output_file = f"{output_directory}Ocfr_{season}_5d_s4max_{s4max_threshold}.nc"
    Ocfr_ds.to_netcdf(output_file)

###########################################################################################################################


Months [3, 4, 5]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:53<00:00, 12991.03it/s]


Months [6, 7, 8]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:49<00:00, 13060.29it/s]


Months [9, 10, 11]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:39<00:00, 13252.85it/s]


Months [12, 1, 2]


100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:45<00:00, 13136.09it/s]


In [31]:
################################       M A G N E T I C Lat Lon coordinates        ##########################################

s4max_threshold = 0.35

#Arras and Wickert 2018 s4max > 0.2
#Yue et al 2016 s4max > 0.3
#Yue et al 2015 s4max > 0.3
#Ray 2006 s4max > 0.4
#Ko and Yeh 2010 'strong scintillations' defined as smax >= 0.5

###########################################################################################################################
# Define multiple sets of desired months
all_desired_months = [ [3, 4, 5], [6, 7, 8], [9, 10, 11] , [12, 1, 2] ]

# Loop over each set of desired months
for desired_months in all_desired_months:
    print('Months ' + str(desired_months))
    averaging_period = ""
    if len(desired_months) == 1:
        averaging_period = "Monthly"
    elif len(desired_months) == 2:
        averaging_period = "Two-Month"
    elif len(desired_months) == 3:
        averaging_period = "Three-Month"
    else:
        averaging_period = "Unknown"

    season = ""
    if set(desired_months) <= {12, 1, 2}:
        season = "winter"
    elif set(desired_months) <= {3, 4, 5}:
        season = "spring"
    elif set(desired_months) <= {6, 7, 8}:
        season = "summer"
    elif set(desired_months) <= {9, 10, 11}:
        season = "autumn"
    else:
        season = "unknown"

    freq_LT_m = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))
    nmeas_LT_m = np.zeros((Zavg_sl_num_bins, lat_num_bins, LT_num_bins))

    freq_lon_m = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))
    nmeas_lon_m = np.zeros((Zavg_sl_num_bins, lat_num_bins, lon_num_bins))

    nfile = len(ds['lat'])
    ds_length = np.arange(nfile)

    ###########################################################################################################################
    for i in tqdm(ds_length):
        current_altitude = ds['altitude_var'].values[i]
        # current_lat = ds['lat'].values[i]
        # current_lon = ds['lon'].values[i]
        current_LT = ds["LT"].values[i]
        current_s4max = ds['s4max'].values[i]
        current_mlat = ds['mlat'].values[i]
        current_mlon = ds['mlon'].values[i]

        current_time = ds["time"].values[i]
        current_month = int(np.datetime64(current_time, 'M').astype('M8[M]').view('i8')) % 12 + 1

        if current_month in desired_months:
            if Zavg_sl_bin_edges[-1] <= current_altitude <= Zavg_sl_bin_edges[0]:
                altitude_bin_index = np.digitize(current_altitude, Zavg_sl_bin_edges, right=False) - 1
                mlat_bin_index = np.digitize(current_mlat, lat_bin_edges) - 1
                mlon_bin_index = np.digitize(current_mlon, lon_bin_edges) - 1
                
                if current_mlon == lon_bin_edges[-1]:
                    mlon_bin_index = len(lon_bin_edges) - 2
                    
                LT_bin_index = np.digitize(current_LT, LT_bin_edges) - 1
                if current_LT == LT_bin_edges[-1]:
                    LT_bin_index = len(LT_bin_edges) - 2

                nmeas_lon_m[altitude_bin_index, mlat_bin_index, mlon_bin_index] += 1.
                nmeas_LT_m[altitude_bin_index, mlat_bin_index, LT_bin_index] += 1.

                if current_s4max > s4max_threshold:
                    freq_lon_m[altitude_bin_index, mlat_bin_index, mlon_bin_index] += 1.
                    freq_LT_m[altitude_bin_index, mlat_bin_index, LT_bin_index] += 1.

    ###########################################################################################################################
    # Calculate occurence frequency
    ocfr_lon_m = np.full((Zavg_sl_num_bins, lat_num_bins, lon_num_bins), 0.)
    ocfr_LT_m = np.full((Zavg_sl_num_bins, lat_num_bins, LT_num_bins), 0.)

    # Check if arrays are equal to 0
    lon_mask_m = (freq_lon_m != 0.) & (nmeas_lon_m != 0.)
    LT_mask_m = (freq_LT_m != 0.) & (nmeas_LT_m != 0.)

    # Calculate ocfr_lon and ocfr_LT only where the mask is True
    if lon_mask_m.any():
        ocfr_lon_m[lon_mask_m] = (freq_lon_m[lon_mask_m] / nmeas_lon_m[lon_mask_m]) * 100.0

    if LT_mask_m.any():
        ocfr_LT_m[LT_mask_m] = (freq_LT_m[LT_mask_m] / nmeas_LT_m[LT_mask_m]) * 100.0

    ###########################################################################################################################
    # Save occurrence frequency to nc file
    Ocfr_ds = xr.Dataset(
        data_vars={
            "ocfr_lon_m": (['altitude', 'magnetic latitude', 'magnetic longitude'], ocfr_lon_m),
            "ocfr_LT_m": (['altitude', 'magnetic latitude', 'local_time'], ocfr_LT_m)
        },
        coords={
            "altitude": Zavg_sl_bin_midpoints,
            "magnetic latitude": lat_bin_midpoints,
            "magnetic longitude": lon_bin_midpoints,
            "local_time": LT_bin_midpoints,
        },
        attrs={
            'averaging_period': averaging_period,
            'season': season
        }
    )

    output_directory = "./Nc_Files/s4max/"
    output_file = f"{output_directory}Ocfr_m_{season}_s4max_{s4max_threshold}.nc"
    if fivedegbins == 1:
        output_file = f"{output_directory}Ocfr_m_{season}_5d_s4max_{s4max_threshold}.nc"
    Ocfr_ds.to_netcdf(output_file)

###########################################################################################################################


Months [3, 4, 5]


100%|█████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:38<00:00, 13260.79it/s]


Months [6, 7, 8]


100%|█████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:39<00:00, 13255.63it/s]


Months [9, 10, 11]


100%|█████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:31<00:00, 13399.58it/s]


Months [12, 1, 2]


100%|█████████████████████████████████████████████████████████████████████| 9266536/9266536 [11:36<00:00, 13299.60it/s]
